Write a query identifying the type of each record in the TRIANGLES table using its three side lengths. Output one of the following statements for each record in the table:

Equilateral: It's a triangle with  sides of equal length.
Isosceles: It's a triangle with  sides of equal length.
Scalene: It's a triangle with  sides of differing lengths.
Not A Triangle: The given values of A, B, and C don't form a triangle.
Input Format

The TRIANGLES table is described as follows:

![Output](https://s3.amazonaws.com/hr-challenge-images/12887/1443815629-ac2a843fb7-1.png)


Each row in the table denotes the lengths of each of a triangle's three sides.

Sample Input

![Output](https://s3.amazonaws.com/hr-challenge-images/12887/1443815827-cbfc1ca12b-2.png)


Sample Output

Isosceles
Equilateral
Scalene
Not A Triangle
Explanation

Values in the tuple  form an Isosceles triangle, because .
Values in the tuple  form an Equilateral triangle, because . Values in the tuple  form a Scalene triangle, because .
Values in the tuple  cannot form a triangle because the combined value of sides  and  is not larger than that of side .

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType

# Initialize Spark session
spark = SparkSession.builder.appName("Triangles").getOrCreate()

# Define schema for TRIANGLES table
schema = StructType([
    StructField("A", IntegerType(), True),
    StructField("B", IntegerType(), True),
    StructField("C", IntegerType(), True)
])

# Create DataFrame with sample triangle side lengths
data = [
    (5, 5, 3),   # Isosceles
    (4, 4, 4),   # Equilateral
    (3, 4, 5),   # Scalene
    (1, 2, 3),   # Not A Triangle
    (7, 10, 5),  # Scalene
    (6, 6, 10)   # Isosceles
]

df = spark.createDataFrame(data, schema=schema)

# Save DataFrame as a temporary table
df.createOrReplaceTempView("TRIANGLES")


In [0]:
%sql
SELECT CASE             
            WHEN A + B > C AND B + C > A AND A + C > B THEN
                CASE 
                    WHEN A = B AND B = C THEN 'Equilateral'
                    WHEN A = B OR B = C OR A = C THEN 'Isosceles'
                    ELSE 'Scalene'
                END
            ELSE 'Not A Triangle'
        END AS Type_of_tringle
FROM TRIANGLES

Type_of_tringle
Isosceles
Equilateral
Scalene
Not A Triangle
Scalene
Isosceles


In [0]:
from pyspark.sql.functions import when
df.selectExpr("""CASE             
            WHEN A + B > C AND B + C > A AND A + C > B THEN
                CASE 
                    WHEN A = B AND B = C THEN 'Equilateral'
                    WHEN A = B OR B = C OR A = C THEN 'Isosceles'
                    ELSE 'Scalene'
                END
            ELSE 'Not A Triangle'
        END AS Type_of_tringle""").show()
            

+---------------+
|Type_of_tringle|
+---------------+
|      Isosceles|
|    Equilateral|
|        Scalene|
| Not A Triangle|
|        Scalene|
|      Isosceles|
+---------------+



In [0]:
from pyspark.sql.functions import when

df = df.withColumn(
    "Type_of_triangle",
    when((df.A == df.B) & (df.B == df.C), "Equilateral")
    .when((df.A + df.B <= df.C) | (df.A + df.C <= df.B) | (df.B + df.C <= df.A), "Not A Triangle")
    .when((df.A == df.B) | (df.B == df.C) | (df.A == df.C), "Isosceles")
    .otherwise("Scalene")
)

df.show()


+---+---+---+----------------+
|  A|  B|  C|Type_of_triangle|
+---+---+---+----------------+
|  5|  5|  3|       Isosceles|
|  4|  4|  4|     Equilateral|
|  3|  4|  5|         Scalene|
|  1|  2|  3|  Not A Triangle|
|  7| 10|  5|         Scalene|
|  6|  6| 10|       Isosceles|
+---+---+---+----------------+

